In [1]:
import pandas as pd
#from top2vec import Top2Vec
import os
import collections
import csv
import logging
import numpy as np
import datetime as datetime
import types

from tensorflow.keras.layers import Input
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Embedding, Concatenate, dot
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.losses import cosine_similarity
from tensorflow.keras.callbacks import TensorBoard
from tensorboard.plugins import projector

In [2]:
df = pd.read_pickle('./Data/df_processed.pickle')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367947 entries, 0 to 367946
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   author             181781 non-null  object        
 1   date               367947 non-null  datetime64[ns]
 2   domain             367947 non-null  object        
 3   title              367862 non-null  object        
 4   url                367947 non-null  object        
 5   content            367947 non-null  object        
 6   topic_area         367947 non-null  object        
 7   content_processed  367947 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 22.5+ MB


In [4]:
df.head(1)

,author,date,domain,title,url,content,topic_area,content_processed
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[end, year, corner, past time, think, position..."


In [5]:
# Note to do - need to add time element

def log_newline(self, how_many_lines=1):
    file_handler = None
    if self.handlers:
        file_handler = self.handlers[0]

    # Switch formatter, output a blank line
    file_handler.setFormatter(self.blank_formatter)
    for i in range(how_many_lines):
        self.info('')

    # Switch back
    file_handler.setFormatter(self.default_formatter)

def logger_w2v():
    
    log_file = os.path.join('./Data', 'word2vec.log')
    print('log file location: ', log_file)
    
    log_format= '%(asctime)s - %(levelname)s - [%(module)s]\t%(message)s'
    formatter = logging.Formatter(fmt=(log_format))
    
    fhandler = logging.FileHandler(log_file)
    fhandler.setFormatter(formatter)
    
    logger = logging.getLogger('word2vec')
    logger.setLevel(logging.DEBUG)
    logger.addHandler(fhandler)
    logger.default_formatter = formatter
    logger.blank_formatter = logging.Formatter(fmt="")
    logger.newline = types.MethodType(log_newline, logger)
    
    return logger
    

In [7]:
class Word2Vec:
    """
    apply word2vec to text
    """

    def __init__(self, logger, vocab_size, vector_dim, input_target, input_context,
                 load_pretrained_weights, checkpoint_file):
        """
        Args:
            vocab size: integer of number of words to form vocabulary from
            vector_dim: integer of number of dimensions per word
            input_target: tensor representing target word
            input_context: tensor representing context word
        """
        self.logger = logger        
        self.vocab_size = vocab_size
        self.vector_dim = vector_dim
        self.input_target = input_target
        self.input_context = input_context
        self.load_pretrained_weights = load_pretrained_weights
        self.checkpoint_file = checkpoint_file
        self.model = self.create_model()
        
    def build_dataset(self, words):
        """
        :process raw inputs into a dataset

        Args:
            words: list of strings

        Returns:
            tuple:
                data: list of integers representing words in words
                count: list of count of most frequent words with size n_words
                dictionary: dictionary of word to unique integer
                reverse dictionary: dictionary of unique integer to word
        """
        self.logger.info("Building dataset")

        count = [['UNK', -1]]
        words = [item for sublist in words for item in sublist]
        count.extend(collections.Counter(words).most_common(self.vocab_size - 1))
        dictionary = dict()
        for word, _ in count:
            dictionary[word] = len(dictionary)
        data = list()
        unk_count = 0
        for word in words:
            if word in dictionary:
                index = dictionary[word]
            else:
                index = 0  # dictionary['UNK']
                unk_count += 1
            data.append(index)
        count[0][1] = unk_count
        reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
        self.dictionary = dictionary

        # Save dictionary
        dict_path = './Data'
        dict_file = 'dictionary.csv'
        dict_file = os.path.join(dict_path,dict_file)
        
        with open(dict_file, 'w') as f:
            for key in dictionary.keys():
                f.write("%s,%s\n"%(key,dictionary[key]))

        return data, count, dictionary, reversed_dictionary
    
    def get_training_data(self, data, window_size):
        """
        :create text and label pairs for model training

        Args:
            data: list of integers representing words in words
            window_size: integer of number of words around the target word that
                         will be used to draw the context words from.

        Returns:
            tuple:
                word_target: list of arrays representing target word 
                word_context: list of arrays representing context word in 
                              relation to target word
                labels: list containing 1 for true context, 0 for false context
        """
        sampling_table = sequence.make_sampling_table(self.vocab_size)
        couples, labels = skipgrams(data, self.vocab_size, window_size=window_size, 
                                    sampling_table=sampling_table)

        word_target, word_context = zip(*couples)
        word_target = np.array(word_target, dtype="int32")
        word_context = np.array(word_context, dtype="int32")

        return word_target, word_context, labels

    def create_model(self):
        """
        :keras functional API and embedding layers

        Returns:
            model: untrained word2vec model
        """

        # embedding layer
        embedding = Embedding(self.vocab_size, self.vector_dim, input_length=1, name='embedding')

        # embedding vectors
        target = embedding(self.input_target)
        target = Reshape((self.vector_dim, 1))(target)
        context = embedding(self.input_context)
        context = Reshape((self.vector_dim, 1))(context)

        # dot product operation to get a similarity measure
        dot_product = dot([target, context], axes=1, normalize=False)
        dot_product = Reshape((1,))(dot_product)

        # add the sigmoid output layer
        output = Dense(1, activation='sigmoid')(dot_product)

        # create the training model
        self.model = Model(inputs=[self.input_target, self.input_context], outputs=output)

        return self.model

    def train_model(self, epochs, batch_size, word_target, word_context, labels):
        """
        :trains word2vec model

        Args:
            model: word2vec model
            epochs: integer of number of iterations to train model on
            batch_size: integer of number of words to pass to epoch
            word_target: list of arrays representing target word 
            word_context: list of arrays representing context word in relation 
                          to target word
            labels: list containing 1 for true context, 0 for false context

        Returns:
            model: trained word2vec model
        """

        self.model.compile(loss='binary_crossentropy', optimizer='rmsprop')

        # tensorboard callback
        current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        log_dir='tensorboard_log/' + current_time
        summary_writer = tf.summary.create_file_writer(log_dir)

        if self.load_pretrained_weights:
            self.load_prior_weights()

        arr_1 = np.zeros((batch_size,))
        arr_2 = np.zeros((batch_size,))
        arr_3 = np.zeros((batch_size,))
        for i in range(epochs):
            idx = np.random.choice(list(range(len(labels))), size=batch_size, replace=False)
            arr_1[:] = np.array([word_target[i] for i in idx])
            arr_2[:] = np.array([word_context[i] for i in idx])
            arr_3[:] = np.array([labels[i] for i in idx])
            loss = self.model.train_on_batch([arr_1, arr_2], arr_3)
            with summary_writer.as_default():
                tf.summary.scalar('loss', loss, step=i)
            if (i+1) % 100 == 0:
                print("Iteration {}, loss={}".format(i+1, loss))
            if (i+1) % 1000 == 0:
                abs_path = os.path.abspath(os.path.join(os.path.dirname( __file__ ), '..'))
                checkpoint_dir = './model/model_weights'
                checkpoint_file = f"cp-epoch-{i+1:010d}.h5"
                checkpoint_path = os.path.join(checkpoint_dir,checkpoint_file)
                self.model.save_weights(checkpoint_path)
                self.embedding_projector(log_dir)

        return self.model

In [8]:
words = df['content_processed'][:100000]
sorted(words[4][:10])

['awards shows',
 'celebrities',
 'coronavirus pandemic',
 'life returns',
 'like',
 'normal',
 'parties',
 'premieres',
 'things',
 'walk red carpets']

In [9]:
logger = logger_w2v()

vocab_size = 10000
vector_dim = 250
input_target = Input((1,))
input_context = Input((1,))
load_pretrained_weights = False
checkpoint_file = None

word2vec = Word2Vec(logger, vocab_size, vector_dim, input_target, input_context,
                    load_pretrained_weights, checkpoint_file)

data, count, dictionary, reversed_dictionary = word2vec.build_dataset(words)

log file location:  ./Data/word2vec.log


In [10]:
print(len(dictionary))
print(len(data))
#count
#reversed_dictionary
#dictionary

10000
25985113


In [11]:
window_size = 3

word_target, word_context, labels = word2vec.get_training_data(data, window_size)

In [ ]:
epochs = 5000
batch_size = 100

logger.info("Training model with {} epochs".format(epochs))

model = word2vec.train_model(epochs, batch_size, word_target, word_context, labels)

Iteration 100, loss=0.6934609413146973
Iteration 200, loss=0.6936301589012146
Iteration 300, loss=0.690919041633606


In [77]:
model

In [78]:
import io
import pandas as pd
import os.path


class Visualization:
    """
    prepare data for visualization in tensorflow
    """

    def __init__(self, logger):
        self.logger = logger


    def tsv_file(self, model, dictionary, visualization_dir):

        embedding_weights = model.layers[2].get_weights()[0]

        word_embeddings = {w:embedding_weights[idx] for w, idx in dictionary.items()}

        out_v_p = os.path.join(visualization_dir, 'vecs.tsv')
        out_m_p = os.path.join(visualization_dir, 'meta.tsv')
        out_v = io.open(out_v_p, 'w', encoding='utf-8')
        out_m = io.open(out_m_p, 'w', encoding='utf-8')

        for num, word in enumerate(word_embeddings):
            vec = embedding_weights[num]
            out_m.write(word + "\n")
            out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        out_v.close()
        out_m.close()
        self.logger.info("files for visualization")
        return


In [79]:
# Visualisation
logger.info("prepare data for visualization")

visualization_prep = Visualization(logger)

visualization_dir = './Visualisation'
visualization_prep.tsv_file(model, dictionary, visualization_dir)